In [1]:
import pandas as pd
import numpy as np
import calendar

from bokeh.charts import output_notebook, Scatter, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.plotting import figure
from bokeh.layouts import row, column, gridplot

In [2]:
import pandas as pd
import numpy as np
import calendar
from bokeh.charts import output_notebook, Scatter, Bar, show, output_file, Line, BoxPlot, Scatter
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.io import hplot
output_notebook() 

Loading BokehJS ...

In [3]:
INPUT="data/device_failure.csv" 
dataset = pd.read_csv(INPUT,index_col=[0,1],parse_dates=[0])

## per device model

 - Set up first model
 - Precision/recall, ROC
 - Calibration
 - PCA ?
 - feature engineering
 - data cleaning
 - Test other models

## Build Training set

In [4]:
# fitering methods
suspicious_positives = set(["S1F0GPFZ", "S1F136J0", "W1F0KCP2", "W1F0M35B", "W1F11ZG9"])
def filter_devices(df):
    return df.filter(axis="index",items=(set(df.index) - suspicious_positives)) 

In [5]:
# feature preprocessing
def build_deriv(df,c,n=1):
    def per_device(per_device):
        clean_index = per_device.reset_index(level=1,drop=True)
        resampled=clean_index.resample('1D',"pad")
        
        raw_diff = np.diff(resampled,n=n)
        #fill the series start with zeros
        while (len(raw_diff) < len(resampled)):
            raw_diff = np.insert(raw_diff,0,0)
        d = pd.Series(data=raw_diff,index = resampled.index )
        #print d
        return d.dropna()[d>0]
    some_d= df[c].groupby(level="device").apply(per_device)
    return some_d.swaplevel()

In [6]:
from scipy.fftpack import fft
#fft_df = feature_dset[[attribute]].copy()

def to_fft(df):
    resampled =  df.resample("1D",level="date").mean().fillna(method='pad')
    n = len(resampled)
    return np.abs(fft(resampled))[n//2:]
def fft_line(df):
    return df.groupby(level="device",sort=True).transform(to_fft)

def peaks(line):
    sorted_by_used = sorted(enumerate(line),key = lambda  t: t[1], reverse=True )
    boundaries = set()
    peaks = []
    for i,value in sorted_by_used:
        if i not in boundaries:
            peaks.append((i,value))
        # in any case, i neighbors cannot be peaks now.
        boundaries.add(i+1)
        boundaries.add(i-1)
    return peaks
            
    
def fft_peak(df,p=0,index_no_value=True):
    fft = to_fft(df)
    #fft = fft_line(df)
    all_peaks = peaks(fft.tolist())
    if (len(all_peaks) > p):
        return all_peaks[p][0 if index_no_value else 1]
    else:
        return 0

In [7]:
def pre_filter(df):
    res = df.copy()
    del res["attribute1"]
    del res["attribute3"]
    #del res["attribute5"]
    dt_list = ["attribute2"]#,"attribute8"]
    for c in dt_list:
        deriv = build_deriv(res,c)
        res["dt_%s" % c] = deriv
        res["dt2_%s" % c] =  build_deriv(res,c,2)
    return res.fillna(0)

def post_filter(df):
    res = df.copy()
    res = filter_devices(res)
    for col in res.columns:
        if "min" in col:
            del res[col]
        if "std" in col:
            del res[col]
    return res

In [8]:
pre_dataset = pre_filter(dataset)
#print feature_set.columns

features = [f for f in pre_dataset.columns if "att" in f]
def f_to_dict(feature):
    d = {
            "min_%s" % feature:np.min,
            "max_%s" % feature:np.max,
            "mean_%s" % feature :np.mean,
            "std_%s" % feature:np.std
        }
    dft_list = ["attribute4","attribute5", "attribute6","attribute7","attribute9"]
    if feature in dft_list:
        d["dft_p0_ind%s" % feature] = lambda r : fft_peak(r,p=0,index_no_value=True)
        d["dft_p0_val%s" % feature] = lambda r : fft_peak(r,p=0,index_no_value=False)
        #d["dft_p1_ind%s" % feature] = lambda r : fft_peak(r,p=1,index_no_value=True)
        #d["dft_p1_val%s" % feature] = lambda r : fft_peak(r,p=1,index_no_value=False)
    return d

agg_dict = dict( (f,f_to_dict(f)) for f in features )
#print agg_dict

/home/avignon/.local/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).pad()


In [32]:
feature_set = pre_dataset.groupby(level="device").agg(agg_dict)
feature_set.columns = feature_set.columns.droplevel()
feature_set = post_filter(feature_set)

# feature filtering
# feature filtering
#try : 
#    filtered = feature_set.filter(items=feature_imp.index)
#    print "filtering devices"
#    feature_set = filtered
#except:
#    print "no feature filtering"

label_set =  dataset[["failure"]].groupby(level="device").sum()
label_set = filter_devices(label_set)
feature_mat = feature_set.as_matrix()
label_mat = label_set.as_matrix().ravel()

## Run model

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import Normalizer
from sklearn.metrics import roc_curve, auc
from sklearn.svm import SVC, NuSVC

#model = GradientBoostingClassifier()
pca = PCA()#n_components="mle",svd_solver="full")
norm = Normalizer()
#model=GradientBoostingClassifier()
model = RandomForestClassifier()
#model = SVC(probability=True)
pipeline= Pipeline([('normalize', norm),('reduce_dim', pca),("model",model)])
try:
    # use best parameters if available
    #
    pipeline.set_params(**grid_result.best_params_)
    print "using last optimized model"
except:
    print "no optim result, or bad ones: let's keep the default ones"
    pass
scores = cross_val_score(pipeline, feature_mat, label_mat,cv=3,verbose=1,scoring="accuracy",n_jobs=6)                          
print "accurracy: %g, std(%g))" % (scores.mean(), scores.std())

no optim result, or bad ones: let's keep the default ones
accurracy: 0.934644, std(0.00685456))


[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    0.1s finished


In [ ]:
pipeline.get_params()

### Eval Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, accuracy_score,precision_recall_curve, auc

X_train, X_test, Y_train, Y_test = train_test_split(feature_mat,label_mat,test_size=0.3)
# calculate the fpr and tpr for all thresholds of the classification

fitted = pipeline.fit(X_train,Y_train)
probs = fitted.predict_proba(X_test)
preds = probs[:,1]
preds_train = fitted.predict_proba(X_train)[:,1]
fpr, tpr, threshold = roc_curve(Y_test, preds)
fpr_train, tpr_train, threshold_train = roc_curve(Y_train, preds_train)
roc_auc = auc(fpr, tpr)
roc_auc_train = auc(fpr_train, tpr_train)
precision, recall, ths = precision_recall_curve(Y_test, preds)
precision_train, recall_train, ths_train = precision_recall_curve(Y_train, preds_train)

In [ ]:
from bokeh.models.ranges import Range1d
#print "auc: %.2g, on train: %.2g" %(roc_auc, roc_auc_train)
roc_df = pd.DataFrame({"fpr":fpr,"tpr":tpr}).set_index("fpr")
pr_df = pd.DataFrame({"precision": precision, "recall":recall}).set_index("recall")
roc_df["diag"] = roc_df.index
pr_df["random"] = pr_df.precision.iloc[0]

# roc curve
roc_f = figure(width=400,height=400,title="roc, auc: %.2g, on train: %.2g"  %(roc_auc, roc_auc_train) )
roc_f.xaxis.axis_label = "tpr"
auc_range= Range1d(0,1)
roc_f.x_range = auc_range 
roc_f.y_range = auc_range 
roc_f.yaxis.axis_label = "fpr"
roc_f.cross(fpr,tpr,size=5)
roc_f.line(fpr,tpr,legend="roc")
roc_f.circle(fpr_train,tpr_train,size=5,color="red", line_width=1)
roc_f.line(fpr_train,tpr_train,color="red",legend="roc on train")
roc_f.line([0,1],[0,1], color="grey")

# pr curve
pr_f = figure(width=400,height=400,title="PR curve")
pr_f.xaxis.axis_label = "recall"
pr_f.yaxis.axis_label = "precision"
pr_f.cross(recall,precision,size=5)
pr_f.line(recall,precision,legend="PR")
pr_f.circle(recall_train,precision_train,size=5,color="red", line_width=1)
pr_f.line(recall_train,precision_train,color="red",legend="PR on train")

show(row(
    pr_f,
    roc_f
))

### Feature Importance

In [ ]:
feature_imp = pd.DataFrame({"importance":model.feature_importances_}).set_index(feature_set.columns)
feature_imp.sort_values(by="importance",ascending=False)

### Hyperparameter optimisation




In [ ]:
# model : GradientBoostingClassifier, parameters: 
#loss : {‘deviance’, ‘exponential’},
#learning_rate : float, optional (default=0.1)
#n_estimators : int (default=100)
#max_depth : integer, optional (default=3)
#min_samples_split : int, float, optional (default=2)
grids=dict()
XDB_param_grid = {
    #"model__loss":  ["deviance", 'exponential'],
    "model__learning_rate" : [1e-3,0.01, 0.1],
    "model__n_estimators" : [10, 50, 100, 150],
    "model__max_depth" : [5,10,15],
    "model__min_samples_split" : [5,10,20]
}
grids[GradientBoostingClassifier] = XDB_param_grid

In [ ]:
# model : RandomForestClassifier, parameters: 
# n_estimators : int (default=100)
# criterion : "gini","entropy"
# max_features : auto , fraction
# max_depth : integer, optional (default=3)
# min_samples_split : int, float, optional (default=2)

RF_param_grid = {
    #"model__criterion":  ["gini", "entropy"],
    "model__n_estimators" : [75,100,150,200],
    #"model__max_features" : ["auto",0.5,0.25,0.1],
    "model__max_depth" : [2,5,10,20],
    "model__min_samples_split" : [5,10,20]
}
grids[RandomForestClassifier] = RF_param_grid

In [ ]:
# C :penalty
# kernel : ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’ 
SVC_param_grid = {
   'model__C': [1e-7,1e-6,1e-5,0.1],
    "model__kernel": ["rbf","linear"],
    #"model_degree" : [1,3,5], # polynomial degrees
    "model__gamma" : ["auto"], # kernel coef (rbf)
    #"coef0" # for poly, signmoid
    "model__tol" : [1e-7,1e-6,1e-5, 1e-4,1e-3]
}
grids[SVC] = SVC_param_grid

In [ ]:
m  = type(dict(pipeline.steps)["model"])
param_grid=grids[m]

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=6, shuffle=True)
grid_search = GridSearchCV(pipeline, param_grid, scoring="accuracy", n_jobs=-1, verbose=1,cv=kfold)
grid_result = grid_search.fit(feature_mat,label_mat)


# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
#for mean, stdev, param in zip(means, stds, params):
#    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
pipeline.get_params()

In [33]:
from tpot import TPOTClassifier
pipeline_optimizer = TPOTClassifier(
    generations=10, # the more generatiion, the more optimized you get
    population_size=200,
    num_cv_folds=4,
    scoring="accuracy",
    random_state=42,
    verbosity=3)

pipeline_optimizer.fit(feature_mat, label_mat)
print pipeline_optimizer.score(feature_mat, label_mat)
pipeline_optimizer.export('tpot_longrun_exported_pipeline.py')

Optimization Progress:   1%|          | 15/2200 [01:23<2:07:14,  3.49s/pipeline]n Progress:   0%|          | 2/2200 [00:01<20:18,  1.80pipeline/s]          Optimization Progress:   1%|          | 16/2200 [01:23<12:03:04, 19.86s/pipeline]

Timeout during evaluation of pipeline #16. Skipping to the next pipeline.


Optimization Progress:   1%|          | 17/2200 [02:23<19:15:34, 31.76s/pipeline]

Timeout during evaluation of pipeline #17. Skipping to the next pipeline.


Optimization Progress:   1%|▏         | 28/2200 [03:49<2:29:27,  4.13s/pipeline]]Optimization Progress:   1%|          | 19/2200 [02:25<9:39:41, 15.95s/pipeline]           Optimization Progress:   1%|▏         | 29/2200 [03:49<8:15:22, 13.69s/pipeline]

...

Timeout during evaluation of pipeline #2056. Skipping to the next pipeline.


Optimization Progress:  96%|█████████▋| 2121/2200 [4:05:18<10:39,  8.10s/pipeline]  Optimization Progress:  94%|█████████▎| 2058/2200 [3:54:56<44:59, 19.01s/pipeline]          Optimization Progress:  96%|█████████▋| 2122/2200 [4:05:18<35:05, 26.99s/pipeline]

Timeout during evaluation of pipeline #2122. Skipping to the next pipeline.


                                                                                  Optimization Progress:  97%|█████████▋| 2124/2200 [4:05:32<20:58, 16.56s/pipeline]

0.994840928633


In [42]:
pipeline_optimizer.export("device_export.py")

In [35]:
model_name = "device"

from sklearn.externals import joblib
import pickle
joblib.dump(pipeline_optimizer._fitted_pipeline, 'model_%s.pkl' % model_name) 
features = feature_set.columns
with open('model_%s.feat' % model_name, "w+") as f: 
    pickle.dump(features,f)

PicklingError: Can't pickle <function <lambda> at 0x7fc3cacef9b0>: it's not found as __main__.<lambda>